In [1]:
from timor.Bodies import Body, Connector, Gender
from timor.Joints import Joint
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
import xml.etree.ElementTree as ET
import numpy as np
from timor.Robot import PinRobot, RobotBase
from pathlib import Path
import pinocchio as pin
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere
from timor.utilities.transformation import Transformation

from timor.utilities.spatial import rotX, rotY, rotZ

2024-12-04 14:34:24,859 Timor INFO Loading custom configurations from /home/jonas/.config/timor.config


In [2]:

density = 1200  # kg / m3 estimated overall material density
inner_diameter = 60 / 1000  # assume the links are hollow, with 5cm thick walls

def cylinder_mass(l: float, r: float, r_inner: float = 0) -> float:
    """Calculates the mass of a (hollow) cylinder"""
    mass = density * l * np.pi * r ** 2
    hollow_mass = density * l * np.pi * r_inner ** 2
    return mass - hollow_mass


def cylinder_inertia(l: float, r: float, r_inner: float = 0):
    """Calculates the inertia of a (hollow) cylinder, assuming a centered coordinate system"""
    mass = cylinder_mass(l, r, r_inner)
    lever = np.asarray([0, 0, 0])
    I = np.zeros((3, 3))
    I[0, 0] = (1 / 12) * mass * (3 * (r_inner ** 2 + r ** 2) + l ** 2)
    I[1, 1] = I[0, 0]
    I[2, 2] = .5 * mass * (r_inner ** 2 + r ** 2)

    return pin.Inertia(mass, lever, I)

In [3]:
from timor.Geometry import Mesh


def create_body_with_stl(name: str, stl_path: str, connector_name: str) -> Body:
    """
    Create a Body object in timor using an STL file for collision and visual geometry.

    Args:
        name (str): Name of the body.
        stl_path (str): Path to the STL file.
        connector_name (str): Name of the connector.

    Returns:
        Body: A Body object with the STL-based geometry.
    """
    # Define the collision and visual geometry using the STL file
    stl_geometry = Mesh({"file": stl_path})



    
    # # Create dummy bodies for demonstration
    diameter = 80 / 1000  # the diameter of our links [m]
    length = 150 / 1000
    # Create the body using the STL geometry
    body = Body(
        body_id=name,
        collision=stl_geometry,
        connectors=[Connector(f'J2_distal+{name}',
                                        Transformation.from_translation([0, 0, length / 2]),
                                        gender=Gender.m,
                                        connector_type='default',
                                        size=[diameter])],
        inertia=cylinder_inertia(length, diameter / 2, inner_diameter / 2)  # Optionally specify inertia if available
    )

    return body

length = 150 / 1000
# Example usage
parent_stl_file_path = "./assets/Assem_4310_BASE/Assem_4310_BASE/meshes/base_motor_link.STL"
child_stl_file_path = "./assets/Assem_4310_BASE/Assem_4310_BASE/meshes/base_out_link.STL"
parent = create_body_with_stl("base_motor_link", parent_stl_file_path, "base_connector")
child = create_body_with_stl("base_out_link", child_stl_file_path, "out")


joint = Joint(
    joint_id='Revolute',
    joint_type='revolute',
    parent_body=parent,
    child_body=child,
    q_limits=np.array([-np.pi, np.pi]),
    torque_limit=1000,
    acceleration_limit=5,
    velocity_limit=10,
    parent2joint=Transformation.from_translation([0, 0, length / 2]) @ Transformation.from_rotation(rotY(np.pi / 2)[:3, :3]),
    joint2child=Transformation.from_translation([0, 0, length / 2])
)

    
# Module header
module_header = ModuleHeader(
    ID="R",
    name="RJ",
    date="2024-12-04",
    author=["Generated by URDF to Timor"],
    email=["support@example.com"],
    affiliation=["Automated Conversion"]
)

# Return as a ModulesDB

timor_modules = ModulesDB({AtomicModule(module_header, [parent, child], [joint])})
timor_modules.debug_visualization()
print(timor_modules)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
ModulesDB({<timor.Module.AtomicModule object at 0x7efda9e8d270>})
